# Generate qsub scripts to compute mean and std per variable

In [1]:
import os
import sys
import yaml
import numpy as np
import xarray as xr

In [2]:
sys.path.insert(0, os.path.realpath('../libs/'))
import preprocess_utils as pu

## 6 hourly residual

In [3]:
config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [4]:
years_range = conf['cache']['years_range']
years = np.arange(years_range[0], years_range[1])

In [6]:
for i_year, year in enumerate(years):
    
    f = open('{}cache_6h_{}.sh'.format(conf['qsub']['qsub_loc'], year), 'w') 
    
    heads = '''#!/bin/bash -l

#PBS -N c{}
#PBS -A NAML0001
#PBS -l walltime=23:59:59
#PBS -l select=1:ncpus=4:mem=364GB
#PBS -q casper
#PBS -o cache_{}.log
#PBS -e cache_{}.err

conda activate credit
cd {}
python write_cache_dataset_6h.py {}
'''.format(year, year, year, 
           conf['qsub']['scripts_loc'], 
           year)
    
    print(heads, file=f)    
    f.close()

f = open('{}batch_write_cache_6h.sh'.format(conf['qsub']['qsub_loc']), 'w')

for i_year, year in enumerate(years):
    print('qsub cache_6h_{}.sh'.format(year), file=f)
    
f.close()